In [99]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
#Webscraping and Data Analysis Libraries
import requests, json

from bs4 import BeautifulSoup

import pandas as pd

from openpyxl import load_workbook

import datetime

import re

from functools import partial

import os

In [2]:
#Plotting Libraries
from bokeh.io import output_notebook, show

from bokeh.models import ColumnDataSource, GMapOptions, HoverTool, TapTool, OpenURL, PanTool, WheelZoomTool

from bokeh.plotting import gmap

API_KEY = os.environ["GOOGL_API_KEY"]

In [9]:
output_notebook()

Loading BokehJS ...

In [32]:
delete_old_search = {"flag": False, "sheet_name": None}

webscrape the page to access all craigslist site URLs

In [3]:
def CalculateMeanPrice(row, search_query):
    
    print("CalculateMEan Price Begin")
    full_url = row['link'] + '/search/sss'
    
    full_query = search_query + " -cracked -replacement -broken -case -cover"
    params = {
    'query': full_query,
    'sort' : 'rel',
    'srchType': 'T',
    'min_price' : '50',
    }
    
    resp = requests.get(full_url, params=params)
    soup = BeautifulSoup(resp.content, "html.parser")
    
    row['link'] = resp.url 
    
    price_gen = map(lambda x:(int(x.text.strip('$')),), soup.select("span.result-meta > span.result-price"))
    df_prices = pd.DataFrame.from_records(price_gen, columns=["price"])
    df_prices = df_prices.loc[df_prices.price < 800]
    
    #print(row['city'])
    row['avg_price'] = round(df_prices.price.mean(), 2)
    #print(df_prices.price.median())
    #print(mean)
    return row
    

In [31]:
def IsSearchQueryLatest(sheet_name, searchquery):
    global delete_old_search
    print("IS srch qry begin")
    if "_" in sheet_name:
        product, search_date = sheet_name.split("_")
    
        if product.upper() == searchquery.upper():
            if (datetime.datetime.now() - datetime.datetime.strptime(search_date, "%d-%m-%y")).days < 2:
                return True
            else:
                delete_old_search["sheet_name"] = sheet_name
                delete_old_search["flag"] = True
    
    return False

In [5]:
def GetPriceList(search_query, file, wb):
    print("Get Price List Begin")
    df_cities = pd.read_excel("Static/CraigsListLinks.xlsx")

    #finding the mean price of product in each CraigsList Site
    df_cities = df_cities.apply(lambda row: CalculateMeanPrice(row, search_query), axis=1)

    writer = pd.ExcelWriter(file, engine="openpyxl")
    writer.book = wb
    sheet_name = search_query.replace(" ", "")+datetime.date.today().strftime("_%d-%m-%y")
    df_cities.to_excel(excel_writer=writer, sheet_name=sheet_name)
    writer.save()
    
    print("Get Price List Done")
    return sheet_name

In [6]:
def ShowPricePlot(file, sheet_name):
    print("Show Price plot Begin")
    df = pd.read_excel(file, sheet_name=sheet_name)

    baseline = df.avg_price.min()-1
    scale = 3
    df["rad"] = df.avg_price.apply(lambda x: (x-baseline)/8)
    

    map_options = GMapOptions(lat=50.5020794, lng=-111.9912878, map_type="roadmap", zoom=3)

    p = gmap(os.environ["GOOGL_API_KEY"], map_options, tools='tap')

    source = ColumnDataSource(
        data=dict(lat=df['lat'].tolist(),
                  lon=df['lon'].tolist(),
                  rad = df['rad'].tolist(),
                  city = df['city'].tolist(),
                  avg_price = df['avg_price'].tolist(),
                  link = df['link'].tolist())
    )



    p.circle(x="lon", y="lat", size="rad", fill_color="green", fill_alpha=0.8, source=source)

    p.add_tools(PanTool(), WheelZoomTool(), HoverTool(
        tooltips=[
            # use @{ } for field names with spaces
            ( 'city', '@city'),
            ('average rate', '$@avg_price')
        ],


        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='mouse'
    ))

    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url="@link")
    show(p)
    print("Show Price Plot Done")

In [30]:
def SearchCraigsListSites(search_query):
    global delete_old_search
    
    delete_old_search["flag"] = False
    delete_old_search["sheet_name"] = None
    
    print("Search CraigsListSites Begin")
    wb = load_workbook("Static/CityPrices.xlsx")
    
    sheet_matched = list(filter(partial(IsSearchQueryLatest, searchquery=search_query.replace(" ", "")), wb.sheetnames))
    
    if sheet_matched:
        sheet_name = sheet_matched[0]
    else:
        if delete_old_search["flag"] is True:
            wb.remove(delete_old_search["sheet_name"])
            wb.save("Static/CityPrices.xlsx")
              
        sheet_name = GetPriceList(search_query, "Static/CityPrices.xlsx", wb)
        
    ShowPricePlot("Static/CityPrices.xlsx", sheet_name)

    print("Search CraigsListSites Done")

In [110]:
#%%timeit
#prices = [result_price.text for result_price in sf_soup.find_all("span", {"class": "result-price"}) if result_price.parent.name == "span"]

In [111]:
#%%timeit
#[result_price.text for result_price in sf_soup.select("span.result-meta > span.result-price")]

In [112]:
#%%timeit
#priceGen1 = map(lambda x:int(x.text.strip('$')), sf_soup.select("span.result-meta > span.result-price"))

In [10]:
SearchCraigsListSites("Iphone 8")

Search CraigsListSites Begin
IS srch qry begin
IS srch qry begin
IS srch qry begin
Show Price plot Begin


Show Price Plot Done
Search CraigsListSites Done
